In [1]:
using Plots
include("coevolution_network_base.jl")
using .CoevolutionNetworkBase
theme(:dracula)
using Profile

In [2]:
# Parameters
L = 40.0
dx = 0.3
x = -L/2:dx:L/2-dx
r = 3.0
M = 15
beta = 2.5
alpha = 1.0
gamma = 0.0
D = 0.001
Nh = 2 * 10^10

# Initialize viral and immune densities
viral_density = zeros(Float64, length(x))
viral_density[Int(round(length(x)/2))] = 100/dx
viral_density2 = zeros(Float64, length(x))
immune_density = zeros(Float64, length(x))

# Create Population instances
population = Population(L, dx, r, M, beta, alpha, gamma, D, Nh, viral_density, immune_density;stochastic=true)
population2 = Population(L, dx, r, M, beta, alpha, gamma, D, Nh, viral_density2, immune_density; stochastic=true)
# populations = [population, population2]
populations = [Population(L, dx, r, M, beta, alpha, gamma, D, Nh, viral_density2, immune_density; stochastic=true) for _ = 1:20] 
println(size(populations,1))

populations[1] = Population(L, dx, r, M, beta, alpha, gamma, D, Nh, viral_density, immune_density;stochastic=true)

# Create Network instance
migration_matrix = 1e-5 * rand(size(populations,1),size(populations,1)) # Define an appropriate migration matrix
println(size(migration_matrix,1))
network = Network(populations, migration_matrix)

# Create Simulation instance
dt = 0.05 # Define an appropriate time step size
duration = 80.0 # Define an appropriate simulation duration
simulation = Simulation(network, dt, duration)

using Profile
@timed begin
    # single_step_evolve!(population,dt)
    run_simulation!(simulation)
end

# Profile.print(format=:flat)
# Profile.clear()
# println(length(simulation.duration_times))
# plot(population.xs, population.viral_density)

20
20

(value = true, time = 10.7027576, bytes = 0, gctime = 0.0, gcstats = Base.GC_Diff(0, 0, 0, 0, 0, 0, 0, 0, 0))

In [ ]:
tot_inf_per_deme = calculate_total_infected_per_deme(simulation)

In [ ]:
p = plot(xlabel="time",ylabel="total infected", yscale=:log10, legend=:none)
for i = 1:size(tot_inf_per_deme,1)
    reg = tot_inf_per_deme[i,:] .> 0
    plot!(p, simulation.duration_times[reg], tot_inf_per_deme[i,reg])
end
display(p)

p = plot(xlabel="antigenic coordinate", ylabel = "density", legend=:none)
for i = 1:size(tot_inf_per_deme,1)
    for j = 1:200:1600
        plot!(p, simulation.trajectory[j].populations[i].xs, simulation.trajectory[j].populations[i].viral_density)
    end
end
display(p)